In [71]:
from glob import glob
from random import choice
from time import time
import os
import h5py
import numpy as np
from random import shuffle

In [48]:



with np.load(r"F:\Project_data\Train_Out\Nepali\Negative\Audio\Nepali_songs_1_segment_18.npz") as data:
             for key, val in data.items():
                print(data[key].shape)

   #  print(data['video3D'].shape)

(128, 1292, 1)
(128, 128, 64, 3)
(11,)


In [49]:
path_to_train = "F:\Project_data\Train_Out"
path_to_test = "F:\Project_data\Test_Out"
npz_data= []


def npzgetdata(path):
    Music_header = ['African','Arabic','Chinese','English','Indian','French','Nepali','Spanish']
    Sent = ["Positive", "Neutral", "Negative"]
    npz_data= []
    for Header in Music_header: 
        h_path = path + os.sep+ Header
#         print(h_path)
#         os.mkdir(h_path)
        for sen in Sent:
            s_path= h_path+ os.sep + sen+os.sep+ "Audio"
            for key in os.listdir(s_path):
                npz_data.append(os.path.join(s_path,key))
    return npz_data



In [50]:
from random import shuffle
npz_data_train =npzgetdata(path_to_train)
npz_data_test = npzgetdata(path_to_test)
shuffle(npz_data_train)
shuffle(npz_data_test)

In [33]:
# print(npz_data_train)
print(npz_data_test)

['F:\\Project_data\\Test_Out\\Indian\\Neutral\\Audio\\Indian_music_11_segment_1.npz', 'F:\\Project_data\\Test_Out\\English\\Negative\\Audio\\English_Music_Negative7.npz', 'F:\\Project_data\\Test_Out\\Indian\\Negative\\Audio\\Indian_songs_22_segment_9.npz', 'F:\\Project_data\\Test_Out\\Chinese\\Neutral\\Audio\\Chinese_Music_14_segment_5.npz', 'F:\\Project_data\\Test_Out\\English\\Neutral\\Audio\\English_Music_Neutral2.npz', 'F:\\Project_data\\Test_Out\\Spanish\\Neutral\\Audio\\Spanish_Music_15_segment_1.npz', 'F:\\Project_data\\Test_Out\\African\\Neutral\\Audio\\African_Music_30_segment_6.npz', 'F:\\Project_data\\Test_Out\\African\\Neutral\\Audio\\African_Music_46_segment_0.npz', 'F:\\Project_data\\Test_Out\\Chinese\\Neutral\\Audio\\Chinese_Music_28_segment_4.npz', 'F:\\Project_data\\Test_Out\\Nepali\\Negative\\Audio\\Nepali_songs_14_segment_8.npz', 'F:\\Project_data\\Test_Out\\French\\Neutral\\Audio\\French_music_69_segment_2.npz', 'F:\\Project_data\\Test_Out\\Nepali\\Neutral\\Audio\\N

In [51]:
def normalize(img):
    '''
    Normalizes an array 
    (subtract mean and divide by standard deviation)
    '''
    eps = 0.001
    if np.std(img) != 0:
        img = (img - np.mean(img)) / np.std(img)
    else:
        img = (img - np.mean(img)) / eps
    return img

In [52]:
class DataReader:
    def __init__(self):
        
#         self.train_files, self.val_files = train_test_split(sorted(npz_data_train), test_size=0.1, random_state=42)
        self.train_files = npz_data_train
        self.val_files =npz_data_test
        print('Training files =', len(self.train_files))
        print('Validation files =', len(self.val_files))

    def next_batch_train(self, batch_size):
        return DataReader._next_batch(batch_size, self.train_files)
     #Count the number of files
    def train_files_count(self):
        return len(self.train_files)
    
    def val_files_count(self):
        return len(self.val_files)
    
    # Get all trainig files (Not used using generator)
    def get_all_training_data(self):
        return DataReader._get_data(self.train_files)
    
    def get_all_validation_data(self):
        return DataReader._get_data(self.val_files)
    
    def generator_train(self, batch_size):
        while True:
            logmel_spc,video_3D, target  = DataReader._next_batch(batch_size, self.train_files)
            yield [video_3D,logmel_spc], [target]
    
    def generator_val(self, batch_size):
        while True:
            logmel_spc,video_3D, target  = DataReader._next_batch(batch_size, self.val_files)
            yield [video_3D,logmel_spc], [target]
    
    #left from here
    @staticmethod
    def _get_data(file_list):
        
        def load_into(filename, logmel_spc,video_3D, target):
            with np.load(filename) as data:
                logmel_spc.append(normalize(data['logmel_spc'][:].astype(np.float32)))#added logmel_spc
                video_3D.append(normalize(data['video3D'][:].astype(np.float32)))
                target.append(data['target'][:].astype(np.float32))
        
        logmel_spc,video_3D, target =[],  [], []#added logmel_spc
        for filename in file_list:
            load_into(filename,logmel_spc,video_3D, target)#added logmel_spc
        
        return np.array(logmel_spc),np.array(video_3D), np.array(target)
    @staticmethod
    def _next_batch(batch_size, file_list):
        return DataReader._get_data([choice(file_list) for _ in range(batch_size)])

                

In [36]:
# if __name__ == '__main__':
#     data_reader = DataReader()
#     a = time()
#     for i in range(5):
#         #print(i)
#         logmel_spc,video_3D, target = data_reader.next_batch_train(10)
#         print('video_3D shape =', video_3D.shape)
        
#     print(time() - a, 'sec')
    
#     print('target =', target)
#     print('video_face.shape =', video_3D.shape)

In [53]:
nb_batch = 3
#Loading the data
data_reader = DataReader()
    
train_generator = data_reader.generator_train(nb_batch)
nb_train_samples = data_reader.train_files_count()
train_steps_per_epoch= int(nb_train_samples // nb_batch)
    
val_generator = data_reader.generator_val(nb_batch)
nb_val_samples = data_reader.val_files_count()
val_steps_per_epoch= int(nb_val_samples // nb_batch)   

Training files = 596
Validation files = 204


In [38]:
# for item in train_generator:
#     print(item)

In [54]:
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, MaxPooling3D, Conv3DTranspose, Conv3D, UpSampling3D, GlobalAveragePooling3D
from tensorflow.keras.layers import MaxPooling2D, Conv2DTranspose, Conv2D, UpSampling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import  BatchNormalization, ReLU, ReLU, Reshape, Activation, Concatenate, Dense
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [57]:
#Ref. https://github.com/avivga/audio-visual-speech-enhancement
#https://machinelearningmastery.com/how-to-implement-major-architecture-innovations-for-convolutional-neural-networks/



#from keras.layers.convolutional import Deconv3D, Deconv2D, ZeroPadding3D

def video_net():
    
    video_shape = (128, 128, 64, 3)
    video_input = Input(shape=video_shape)
    nb_neurons = 16
    
    #******************************************** START Video BRANCH *******************************************************
    # BLOCK 1: Video Main
    conv_x1v = Conv3D(filters=nb_neurons, kernel_size=(1, 5, 5), padding='same', activation='relu', kernel_initializer='he_normal')(video_input)
    conc1xv = Concatenate(axis=4)([video_input, conv_x1v])   
    conv_y1v = Conv3D(filters=1, kernel_size=(5, 5, 1), padding='same', activation='relu', kernel_initializer='he_normal')(conc1xv)
    conc1yv = Concatenate(axis=4)([video_input, conv_y1v])
    conv_z1v = Conv3D(filters=nb_neurons, kernel_size=(1, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(conc1yv)
    conc1zv = Concatenate(axis=4)([video_input, conv_z1v])
    mp_z1v = MaxPooling3D(pool_size=(2,2, 2), padding='same')(conc1zv)
    st_y1v = Conv3D(filters=nb_neurons, kernel_size=(3, 3, 1), padding='same', activation='relu', kernel_initializer='he_normal')(video_input)
    conc1stv = Concatenate(axis=4)([video_input, st_y1v]) 
    mp_st1v = MaxPooling3D(pool_size=(2,2,2), padding='same')(conc1stv)
    out_1v = add([mp_st1v, mp_z1v])
    out_c1v = Conv3D(filters=nb_neurons*2, kernel_size=(3, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(out_1v)
    out_c1vx = BatchNormalization()(out_c1v)
    
    
    # BLOCK 2:Video Main
    conv_x2v = Conv3D(filters=nb_neurons*2, kernel_size=(1, 5, 5), padding='same', activation='relu', kernel_initializer='he_normal')(out_c1vx)
    conc2xv = Concatenate(axis=4)([out_c1vx, conv_x2v])
    conv_y2v = Conv3D(filters=1, kernel_size=(5, 5, 1), padding='same', activation='relu', kernel_initializer='he_normal')(conc2xv)
    conc2yv = Concatenate(axis=4)([out_c1vx, conv_y2v])
    conv_z2v = Conv3D(filters=nb_neurons*2, kernel_size=(1, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(conc2yv)
    conc2zv = Concatenate(axis=4)([out_c1vx, conv_z2v])
    mp_z2v = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conc2zv)
    st_y2v = Conv3D(filters=nb_neurons*2, kernel_size=(3, 3, 1), padding='same', activation='relu', kernel_initializer='he_normal')(out_c1vx)
    conc2stv = Concatenate(axis=4)([out_c1vx, st_y2v])
    mp_st2v = MaxPooling3D(pool_size=(2,2,2), padding='same')(conc2stv)
    out_2v = add([mp_st2v, mp_z2v])
    out_c2v = Conv3D(filters=nb_neurons*4, kernel_size=(3, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(out_2v)
    out_c2vx = BatchNormalization()(out_c2v)
    
         
    # BLOCK 3: Video Main
    conv_x3v = Conv3D(filters=nb_neurons*4, kernel_size=(1, 5, 5), padding='same', activation='relu', kernel_initializer='he_normal')(out_c2vx)
    conc3xv = Concatenate(axis=4)([out_c2vx, conv_x3v])
    conv_y3v = Conv3D(filters=1, kernel_size=(5, 5, 1), padding='same', activation='relu', kernel_initializer='he_normal')(conc3xv)
    conc3yv = Concatenate(axis=4)([out_c2vx, conv_y3v])
    conv_z3v = Conv3D(filters=nb_neurons*4, kernel_size=(1, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(conc3yv)
    conc3zv = Concatenate(axis=4)([out_c2vx, conv_z3v])
    mp_z3v = MaxPooling3D(pool_size=(2,2,2), padding='same')(conc3zv)
    st_y3v = Conv3D(filters=nb_neurons*4, kernel_size=(3, 3, 1), padding='same', activation='relu', kernel_initializer='he_normal')(out_c2vx)
    conc3stv = Concatenate(axis=4)([out_c2vx, st_y3v])
    mp_st3v = MaxPooling3D(pool_size=(2,2,2), padding='same')(conc3stv)
    out_3v = add([mp_st3v, mp_z3v])
    out_c3v = Conv3D(filters=nb_neurons*8, kernel_size=(3, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(out_3v)
    out_c3vx = BatchNormalization()(out_c3v)
    
       
    # BLOCK 4: Video Main
    conv_x4v = Conv3D(filters=nb_neurons*8, kernel_size=(1, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(out_c3vx)
    conc4xv = Concatenate(axis=4)([out_c3vx, conv_x4v])
    conv_y4v = Conv3D(filters=1, kernel_size=(3, 3, 1), padding='same', activation='relu', kernel_initializer='he_normal')(conc4xv)
    conc4yv = Concatenate(axis=4)([out_c3vx, conv_y4v])
    conv_z4v = Conv3D(filters=nb_neurons*8, kernel_size=(1, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(conc4yv)
    conc4zv = Concatenate(axis=4)([out_c3vx, conv_z4v])
    mp_z4v = MaxPooling3D(pool_size=(2,2,2), padding='same')(conc4zv)
    st_y4v = Conv3D(filters=nb_neurons*8, kernel_size=(3, 3, 1), padding='same', activation='relu', kernel_initializer='he_normal')(out_c3vx)
    conc4stv = Concatenate(axis=4)([out_c3vx, st_y4v])
    mp_st4v = MaxPooling3D(pool_size=(2,2,2), padding='same')(conc4stv)
    out_4v = add([mp_st4v, mp_z4v])
    out_c4v = Conv3D(filters=nb_neurons*16, kernel_size=(3, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(out_4v)
    out_c4vx = BatchNormalization()(out_c4v)
    
      
    # BLOCK 5: Video Main
    conv_x5v = Conv3D(filters=nb_neurons*16, kernel_size=(1, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(out_c4vx)
    conc5xv = Concatenate(axis=4)([out_c4vx, conv_x5v])
    conv_y5v = Conv3D(filters=1, kernel_size=(3, 3, 1), padding='same', activation='relu', kernel_initializer='he_normal')(conc5xv)
    conc5yv = Concatenate(axis=4)([out_c4vx, conv_y5v])
    conv_z5v = Conv3D(filters=nb_neurons*16, kernel_size=(1, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(conc5yv)
    conc5zv = Concatenate(axis=4)([out_c4vx, conv_z5v])
    mp_z5v = MaxPooling3D(pool_size=(2,2,2), padding='same')(conc5zv)
    st_y5v = Conv3D(filters=nb_neurons*16, kernel_size=(3, 3, 1), padding='same', activation='relu', kernel_initializer='he_normal')(out_c4vx)
    conc5stv = Concatenate(axis=4)([out_c4vx, st_y5v])
    mp_st5v = MaxPooling3D(pool_size=(2,2,2), padding='same')(conc5stv)
    out_5v = add([mp_st5v, mp_z5v])
    out_c5v = Conv3D(filters=nb_neurons*32, kernel_size=(3, 3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(out_5v)
    out_c5vx = BatchNormalization()(out_c5v)
    
    
    #******************************MUSIC 2D Network (GLOCAL Net) **********************************************************#
    audio_shape_phase = (128, 1292, 1)
    nb_neurons = 16
    activation="sigmoid"

    audio_input_mag = Input(shape=audio_shape_phase)
     
    # BLOCK 1: AUDIO Mag
    conv_x1a = Conv2D(filters= nb_neurons, kernel_size=(1, 5), padding='same', activation=activation, kernel_initializer='he_normal')(audio_input_mag)
    conc1xa = Concatenate(axis=3)([audio_input_mag, conv_x1a])
    conv_y1a = Conv2D(filters=1, kernel_size=(5, 1), padding='same', activation=activation, kernel_initializer='he_normal')(conc1xa)
    conc1ya = Concatenate(axis=3)([audio_input_mag, conv_y1a])
    conv_z1a = Conv2D(filters=nb_neurons, kernel_size=(1, 3), padding='same', activation=activation, kernel_initializer='he_normal')(conc1ya)
    conc1za = Concatenate(axis=3)([audio_input_mag, conv_z1a])
    mp_z1a = MaxPooling2D(pool_size=(2,2), padding='same')(conc1za)
    st_y1a = Conv2D(filters=nb_neurons, kernel_size=(3, 1), padding='same', activation=activation, kernel_initializer='he_normal')(audio_input_mag)
    conc1sta = Concatenate(axis=3)([audio_input_mag, st_y1a])
    mp_st1a = MaxPooling2D(pool_size=(2,2), padding='same')(conc1sta)
    out_1a = add([mp_st1a, mp_z1a])
    out_c1a = Conv2D(filters=nb_neurons*2, kernel_size=(3, 3), padding='same', activation=activation, kernel_initializer='he_normal')(out_1a)
    out_c1ax = BatchNormalization()(out_c1a)
    
        # BLOCK 2: AUDIO Mag
    conv_x2a = Conv2D(filters=nb_neurons*2, kernel_size=(1, 5), padding='same', activation=activation, kernel_initializer='he_normal')(out_c1ax)
    conc2xa = Concatenate(axis=3)([out_c1ax, conv_x2a])
    conv_y2a = Conv2D(filters=1, kernel_size=(5, 1), padding='same', activation=activation, kernel_initializer='he_normal')(conc2xa)
    conc2ya = Concatenate(axis=3)([out_c1ax, conv_y2a])
    conv_z2a = Conv2D(filters=nb_neurons*2, kernel_size=(1, 3), padding='same', activation=activation, kernel_initializer='he_normal')(conc2ya)
    conc2za = Concatenate(axis=3)([out_c1ax, conv_z2a])
    mp_z2a = MaxPooling2D(pool_size=(2,2), padding='same')(conc2za)
    st_y2a = Conv2D(filters=nb_neurons*2, kernel_size=(3, 1), padding='same', activation=activation,kernel_initializer='he_normal')(out_c1ax)
    conc2sta = Concatenate(axis=3)([out_c1ax, st_y2a])
    mp_st2a = MaxPooling2D(pool_size=(2,2), padding='same')(conc2sta)    
    out_2a = add([mp_st2a, mp_z2a])
    out_c2a = Conv2D(filters=nb_neurons*4, kernel_size=(3, 3), padding='same', activation=activation, kernel_initializer='he_normal')(out_2a)
    out_c2ax = BatchNormalization()(out_c2a)
    
    # BLOCK 3: AUDIO Mag
    conv_x3a = Conv2D(filters=nb_neurons*4, kernel_size=(1, 5), padding='same', activation=activation, kernel_initializer='he_normal')(out_c2ax)
    conc3xa = Concatenate(axis=3)([out_c2ax, conv_x3a])
    conv_y3a = Conv2D(filters=1, kernel_size=(5, 1), padding='same', activation=activation, kernel_initializer='he_normal')(conc3xa)
    conc3ya = Concatenate(axis=3)([out_c2ax, conv_y3a])
    conv_z3a = Conv2D(filters=nb_neurons*4, kernel_size=(1, 3), padding='same', activation=activation, kernel_initializer='he_normal')(conc3ya)
    conc3za = Concatenate(axis=3)([out_c2ax, conv_z3a])
    mp_z3a = MaxPooling2D(pool_size=(2,2), padding='same')(conc3za)
    st_y3a = Conv2D(filters=nb_neurons*4, kernel_size=(3, 1), padding='same', activation=activation, kernel_initializer='he_normal')(out_c2ax)
    conc3sta = Concatenate(axis=3)([out_c2ax, st_y3a])
    mp_st3a = MaxPooling2D(pool_size=(2,2), padding='same')(conc3sta)
    out_3a = add([mp_st3a, mp_z3a])
    out_c3a = Conv2D(filters=nb_neurons*8, kernel_size=(3, 3), padding='same', activation=activation, kernel_initializer='he_normal')(out_3a)
    out_c3ax = BatchNormalization()(out_c3a)
    
    # BLOCK 4: AUDIO Mag
    conv_x4a = Conv2D(filters=nb_neurons*8, kernel_size=(1, 3), padding='same', activation=activation, kernel_initializer='he_normal')(out_c3ax)
    conc4xa = Concatenate(axis=3)([out_c3ax, conv_x4a])
    conv_y4a = Conv2D(filters=1, kernel_size=(3, 1), padding='same', activation=activation, kernel_initializer='he_normal')(conc4xa)
    conc4ya = Concatenate(axis=3)([out_c3ax, conv_y4a])
    conv_z4a = Conv2D(filters=nb_neurons*8, kernel_size=(1, 3), padding='same', activation=activation, kernel_initializer='he_normal')(conc4ya)
    conc4za = Concatenate(axis=3)([out_c3ax, conv_z4a])
    mp_z4a = MaxPooling2D(pool_size=(2,2), padding='same')(conc4za)
    st_y4a = Conv2D(filters=nb_neurons*8, kernel_size=(3, 1), padding='same', activation=activation, kernel_initializer='he_normal')(out_c3ax)
    conc4sta = Concatenate(axis=3)([out_c3ax, st_y4a])
    mp_st4a = MaxPooling2D(pool_size=(2,2), padding='same')(conc4sta)
    out_4a = add([mp_st4a, mp_z4a])
    out_c4a = Conv2D(filters=nb_neurons*16, kernel_size=(3, 3), padding='same', activation=activation, kernel_initializer='he_normal')(out_4a)
    out_c4ax = BatchNormalization()(out_c4a)
    
    # BLOCK 5: AUDIO Mag
    conv_x5a = Conv2D(filters=nb_neurons*16, kernel_size=(1, 3), padding='same', activation=activation, kernel_initializer='he_normal')(out_c4ax)
    conc5xa = Concatenate(axis=3)([out_c4ax, conv_x5a])
    conv_y5a = Conv2D(filters=1, kernel_size=(3, 1), padding='same', activation=activation, kernel_initializer='he_normal')(conc5xa)
    conc5ya = Concatenate(axis=3)([out_c4ax, conv_y5a])
    conv_z5a = Conv2D(filters=nb_neurons*16, kernel_size=(1, 3), padding='same', activation=activation, kernel_initializer='he_normal')(conc5ya)
    conc5za = Concatenate(axis=3)([out_c4ax, conv_z5a])
    mp_z5a = MaxPooling2D(pool_size=(2,2), padding='same')(conc5za)
    st_y5a = Conv2D(filters=nb_neurons*16, kernel_size=(3, 1), padding='same', activation=activation, kernel_initializer='he_normal')(out_c4ax)
    conc5sta = Concatenate(axis=3)([out_c4ax, st_y5a])
    mp_st5a = MaxPooling2D(pool_size=(2,2), padding='same')(conc5sta)
    out_5a = add([mp_st5a, mp_z5a])
    out_c5a = Conv2D(filters=nb_neurons*32, kernel_size=(3, 3), padding='same', activation=activation, kernel_initializer='he_normal')(out_5a)
    out_c5ax = BatchNormalization()(out_c5a)
    
    
    #********************************************* END MUSIC NETWORK ******************************************************#
    gap_2D_Audio = GlobalAveragePooling2D()(out_c5ax)
    gap_3D_video = GlobalAveragePooling3D()(out_c5vx)
 
    final_out = concatenate([gap_3D_video, gap_2D_Audio])
    final_out = Dense(11, activation='sigmoid', name='AV_OUT')(final_out)
    
    model = Model(inputs=[video_input, audio_input_mag], outputs=[final_out])
    model.summary()
    
    #model.load_weights('/VideoMMTM_Music2D_model.h5', by_name = True)
    #print("The pre-trained weight loaded.")
    
    model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(learning_rate=1e-3),metrics=['categorical_accuracy'])  

    return model
    


In [58]:
AV_model = video_net()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 128, 128, 64 0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 128, 1292, 1 0                                            
__________________________________________________________________________________________________
conv3d_100 (Conv3D)             (None, 128, 128, 64, 1216        input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_100 (Conv2D)             (None, 128, 1292, 16 96          input_10[0][0]                   
____________________________________________________________________________________________

In [72]:
plot_model(AV_model,to_file='F:\Project_data\Code\Av_Model.png',show_shapes=True)

# from ann_visualizer.visualize import ann_viz
# import graphviz

# # Usage
# ann_viz(AV_model,filename='AV_model',title='AV_model')
# graph_file = graphviz.Source.from_file('AV_model.gv')
# graph_file

AttributeError: module 'os' has no attribute 'errno'

In [68]:
# import visualkeras

# visualkeras.layered_view(AV_model, legend=True, to_file='AV_model.png').show() # without custom font



In [43]:

AV_model.fit(train_generator, steps_per_epoch=train_steps_per_epoch, epochs=50,
                    validation_data=val_generator, validation_steps=val_steps_per_epoch,verbose = 1)

Epoch 1/50
198/198 [==============================] - 121s 596ms/step - loss: 4.2912 - categorical_accuracy: 0.1616 - val_loss: 5.0015 - val_categorical_accuracy: 0.0392
Epoch 2/50
198/198 [==============================] - 179s 905ms/step - loss: 4.0259 - categorical_accuracy: 0.1599 - val_loss: 4.8575 - val_categorical_accuracy: 0.1765
Epoch 3/50
198/198 [==============================] - 128s 643ms/step - loss: 3.8768 - categorical_accuracy: 0.2003 - val_loss: 4.1672 - val_categorical_accuracy: 0.1324
Epoch 4/50
198/198 [==============================] - 108s 546ms/step - loss: 3.9776 - categorical_accuracy: 0.2407 - val_loss: 5.3964 - val_categorical_accuracy: 0.1422
Epoch 5/50
198/198 [==============================] - 173s 875ms/step - loss: 3.9795 - categorical_accuracy: 0.2761 - val_loss: 4.9831 - val_categorical_accuracy: 0.2353
Epoch 6/50
198/198 [==============================] - 139s 701ms/step - loss: 3.9848 - categorical_accuracy: 0.3519 - val_loss: 5.2806 - val_categoric